<a href="https://colab.research.google.com/github/BVika/Machine_learning_methods/blob/main/%D0%9B%D0%B0%D0%B1_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задания
Реализовать GPT как в п.2

## Установка библиотек

In [2]:
!pip install nltk
!pip install pymorphy3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 17.7 MB/s eta 0:00:00


## Импорт Библиотек и Загрузка Данных

In [3]:
import nltk
from nltk.stem import SnowballStemmer
from pymorphy3 import MorphAnalyzer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
import math

nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## Текст(датабейс)

In [4]:
input_text = 'Солнце медленно поднималось над горизонтом, окрашивая небо в яркие оттенки оранжевого и розового. Птицы начали петь, наполняя утро мелодиями.'

phrases = [
    'Солнце поднималось',
    'Небо окрашивалось',
    'Птицы начали',
    'Утро наполнилось',
    'Горизонт был',
    'Мир просыпался',
    'Звуки природы',
    'Время летело'
    ]

result = [
    'горизонтом',
    'в краски',
    'петь',
    'мелодиями',
    'красивым',
    'новыми',
    'звучали',
    'быстро'
    ]

In [8]:
class TextProcessor:
    def split_text(self, content: str) -> list[str]:
        """Токенизация текста."""
        return word_tokenize(content)

    def normalize_words(self, tokens: list[str]) -> list[str]:
        """Лемматизация токенов."""
        morph = MorphAnalyzer()
        return [morph.parse(word)[0].normal_form for word in tokens]

    def reduce_words(self, tokens: list[str]) -> list[str]:
        """Стемминг токенов."""
        stemmer = SnowballStemmer("russian")
        return [stemmer.stem(word) for word in tokens]

    def create_vector(self, tokens: list[str]) -> list[int]:
        """Создание векторного представления слов."""
        vector_dict = {}
        vector_result = []
        for word in tokens:
            if word not in vector_dict:
                vector_dict[word] = len(vector_dict)
            vector_result.append(vector_dict[word])
        return vector_result

    def filter_stopwords(self, tokens: list[str]) -> list[str]:
        """Удаление стоп-слов из токенов."""
        stop_words = set(stopwords.words('russian')).union(['.', ',', ':', '?', '!'])
        return [word for word in tokens if word not in stop_words]

    def word_frequency(self, tokens: list[str]) -> dict[str, int]:
        """Создание словаря частот слов."""
        freq_dict = {}
        for word in tokens:
            freq_dict[word] = freq_dict.get(word, 0) + 1
        return freq_dict

    def term_frequency(self, tokens: list[str]) -> dict[str, float]:
        """Расчет частоты термина (TF)."""
        freq_dict = self.word_frequency(tokens)
        total_tokens = len(tokens)
        return {word: count / total_tokens for word, count in freq_dict.items()}

    def inverse_document_frequency(self, documents: list[list[str]]) -> dict[str, float]:
        """Расчет обратной частоты документа (IDF)."""
        idf_dict = {}
        all_words = set(word for doc in documents for word in set(doc))
        total_docs = len(documents)
        for word in all_words:
            idf_dict[word] = math.log(total_docs / sum(word in doc for doc in documents))
        return idf_dict

    def tf_idf(self, documents: list[list[str]], doc_index: int) -> dict[str, float]:
        """Расчет TF-IDF для заданного документа."""
        tf = self.term_frequency(documents[doc_index])
        idf = self.inverse_document_frequency(documents)
        return {word: tf[word] * idf[word] for word in tf}


In [9]:

class CustomTransformer:
    def __init__(self, vocab_size, embed_size, hidden_size, learning_rate=0.01):
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.learning_rate = learning_rate

        # Инициализация параметров модели
        self.embeddings = np.random.randn(vocab_size, embed_size) * 0.01
        self.W_query = np.random.randn(embed_size, hidden_size) * 0.01
        self.W_key = np.random.randn(embed_size, hidden_size) * 0.01
        self.W_value = np.random.randn(embed_size, hidden_size) * 0.01
        self.W_output = np.random.randn(hidden_size, vocab_size) * 0.01


    def softmax(self, x):
        """Применение функции softmax."""
        exp_x = np.exp(x - np.max(x))
        return exp_x / np.sum(exp_x)

    def forward_pass(self, input_indices):
        """Прямой проход через модель."""
        self.input_indices = input_indices
        self.embedded_input = self.embeddings[input_indices]  # (seq_len, embed_size)

        self.Q = self.embedded_input @ self.W_query  # (seq_len, hidden_size)
        self.K = self.embedded_input @ self.W_key
        self.V = self.embedded_input @ self.W_value
        self.attention_scores = self.Q @ self.K.T / np.sqrt(self.embed_size)  # (seq_len, seq_len)
        self.attention_weights = self.softmax(self.attention_scores)  # (seq_len, seq_len)

        self.attended_values = self.attention_weights @ self.V  # (seq_len, hidden_size)
        self.context_vector = np.mean(self.attended_values, axis=0)  # (hidden_size,)

        self.logits = self.context_vector @ self.W_output  # (vocab_size,)
        self.probabilities = self.softmax(self.logits)

        return self.probabilities

    def backward_pass(self, target_index):
        """Обратный проход для обновления весов."""
        d_logits = self.probabilities.copy()
        d_logits[target_index] -= 1  # (vocab_size,)

        # Расчет градиентов
        dW_output = np.outer(self.context_vector, d_logits)  # (hidden_size, vocab_size)
        d_context = d_logits @ self.W_output.T  # (hidden_size,)

        d_attended_values = np.ones_like(self.attended_values) * d_context / self.attended_values.shape[0]  # (seq_len, hidden_size)
        dV = self.attention_weights.T @ d_attended_values  # (seq_len, hidden_size)

        d_attention_weights = d_attended_values @ self.V.T  # (seq_len, seq_len)
        d_scores = d_attention_weights * self.attention_weights * (1 - self.attention_weights)  # (seq_len, seq_len)

        dQ = d_scores @ self.K / np.sqrt(self.embed_size)
        dK = d_scores.T @ self.Q / np.sqrt(self.embed_size)

        # Градиенты для весов
        dW_query = self.embedded_input.T @ dQ
        dW_key = self.embedded_input.T @ dK
        dW_value = self.embedded_input.T @ dV

        # Градиенты для эмбеддингов
        d_embedding = dQ @ self.W_query.T + dK @ self.W_key.T + dV @ self.W_value.T  # (seq_len, embed_size)

        for i, idx in enumerate(self.input_indices):
            self.embeddings[idx] -= self.learning_rate * d_embedding[i]

        # Обновление весов
        self.W_output -= self.learning_rate * dW_output
        self.W_query -= self.learning_rate * dW_query
        self.W_key -= self.learning_rate * dW_key
        self.W_value -= self.learning_rate * dW_value

    def train_step(self, input_indices, target_index):
        """Шаг обучения."""
        self.forward_pass(input_indices)
        self.backward_pass(target_index)

    def predict(self, input_indices):
        """Предсказание на основе входных индексов."""
        probabilities = self.forward_pass(input_indices)
        return np.argmax(probabilities)


In [15]:
# Обработка текста и подготовка данных
text_processor = TextProcessor()


In [16]:
# Обработка текстов и создание векторов
tokens = text_processor.filter_stopwords(text_processor.normalize_words(text_processor.split_text(input_text)))

# Векторизация словаря
vocabulary = list(set(tokens))
word_to_index = {word: i for i, word in enumerate(vocabulary)}
index_to_word = {i: word for word, i in word_to_index.items()}


In [18]:
# Подготовка данных для обучения

X = []
y = []

for phrase, label in zip(phrases, result):
    print(f"Обрабатываемая фраза: {phrase}, метка: {label}")  # Выводим фразы и метки
    tokens = text_processor.filter_stopwords(text_processor.normalize_words(text_processor.split_text(phrase)))
    print(f"Токены после фильтрации: {tokens}")  # Выводим токены после фильтрации

    if len(tokens) < 2:
        print("Пропускаем фразу из-за недостаточной длины.")  # Сообщение о пропуске
        continue  # Пропускаем фразы с недостаточным количеством токенов

    input_indices = [word_to_index.get(word, -1) for word in tokens if word in word_to_index]
    input_indices = [index for index in input_indices if index != -1]  # Удаляем недопустимые индексы

    if len(input_indices) == 0:
        print("Пропускаем фразу из-за отсутствия допустимых токенов.")  # Сообщение о пропуске
        continue  # Пропускаем пустые массивы

    label_index = word_to_index.get(label, -1)
    if label_index == -1:
        print("Пропускаем фразу из-за недопустимой метки.")  # Сообщение о пропуске
        continue  # Пропускаем, если метка недопустима

    X.append(input_indices)
    y.append(label_index)



Обрабатываемая фраза: Солнце поднималось, метка: горизонтом
Токены после фильтрации: ['солнце', 'подниматься']
Пропускаем фразу из-за недопустимой метки.
Обрабатываемая фраза: Небо окрашивалось, метка: в краски
Токены после фильтрации: ['небо', 'окрашиваться']
Пропускаем фразу из-за недопустимой метки.
Обрабатываемая фраза: Птицы начали, метка: петь
Токены после фильтрации: ['птица', 'начать']
Обрабатываемая фраза: Утро наполнилось, метка: мелодиями
Токены после фильтрации: ['утро', 'наполниться']
Пропускаем фразу из-за недопустимой метки.
Обрабатываемая фраза: Горизонт был, метка: красивым
Токены после фильтрации: ['горизонт']
Пропускаем фразу из-за недостаточной длины.
Обрабатываемая фраза: Мир просыпался, метка: новыми
Токены после фильтрации: ['мир', 'просыпаться']
Пропускаем фразу из-за отсутствия допустимых токенов.
Обрабатываемая фраза: Звуки природы, метка: звучали
Токены после фильтрации: ['звук', 'природа']
Пропускаем фразу из-за отсутствия допустимых токенов.
Обрабатываемая 

In [19]:
# Проверка размеров
print(f"Количество примеров: {len(X)}, X: {X}, y: {y}")

Количество примеров: 1, X: [[1, 11]], y: [15]


In [20]:
# Проверка на пустые массивы перед обучением
if len(X) == 0 or len(y) == 0:
    print("Ошибка: нет данных для обучения. Проверьте входные данные.")
else:
    # Обучение модели
    model = CustomTransformer(vocab_size=len(vocabulary), embed_size=16, hidden_size=32)

    for epoch in range(10000):
        total_loss = 0
        for i in range(len(X)):
            model.train_step(X[i], y[i])
            probs = model.forward_pass(X[i])
            total_loss += -np.log(probs[y[i]] + 1e-9)
        if epoch % 1000 == 0:
            print(f"Epoch {epoch}, Loss: {total_loss / len(X)}")

Epoch 0, Loss: 2.7725827325074905
Epoch 1000, Loss: 2.7725672739214797
Epoch 2000, Loss: 2.7725413257824734
Epoch 3000, Loss: 2.7724869017732385
Epoch 4000, Loss: 2.772344406388924
Epoch 5000, Loss: 2.771822786448161
Epoch 6000, Loss: 2.7679674992366503
Epoch 7000, Loss: 2.123479707462627
Epoch 8000, Loss: 0.006003902897369979
Epoch 9000, Loss: 0.002327604353827304


In [21]:
    # Предсказание
    print("\nПредсказания для новых фраз:")
    new_phrases = [
        'Солнце поднималось над горизонтом',
        'Небо стало ярким',
        'Птицы пели мелодии',
        'Утро было прекрасным'
    ]

    for phrase in new_phrases:
        print(f"\nОбрабатываемая новая фраза: '{phrase}'")
        tokens = text_processor.filter_stopwords(text_processor.normalize_words(text_processor.split_text(phrase)))
        print(f"Токены после фильтрации: {tokens}")

        input_indices = [word_to_index.get(word, -1) for word in tokens if word in word_to_index]
        input_indices = [index for index in input_indices if index != -1]  # Удаляем недопустимые индексы

        if len(input_indices) == 0:
            print("Нет допустимых токенов для предсказания.")
            continue

        predicted_index = model.predict(input_indices)
        predicted_word = index_to_word.get(predicted_index, "Неизвестное слово")
        print(f"Предсказанное слово: '{predicted_word}'")


Предсказания для новых фраз:

Обрабатываемая новая фраза: 'Солнце поднималось над горизонтом'
Токены после фильтрации: ['солнце', 'подниматься', 'горизонт']
Предсказанное слово: 'петь'

Обрабатываемая новая фраза: 'Небо стало ярким'
Токены после фильтрации: ['небо', 'стать', 'яркий']
Предсказанное слово: 'петь'

Обрабатываемая новая фраза: 'Птицы пели мелодии'
Токены после фильтрации: ['птица', 'петь', 'мелодия']
Предсказанное слово: 'петь'

Обрабатываемая новая фраза: 'Утро было прекрасным'
Токены после фильтрации: ['утро', 'прекрасный']
Предсказанное слово: 'птица'


In [22]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer

# Ensure to download the necessary NLTK data files
nltk.download('punkt')
nltk.download('wordnet')

class TextProcessor:
    def __init__(self):
        self.lemmatizer = WordNetLemmatizer()
        self.vectorizer = None

    def preprocess(self, text):
        tokens = nltk.word_tokenize(text.lower())
        return [self.lemmatizer.lemmatize(token) for token in tokens if token.isalnum()]

    def fit_vectorizer(self, corpus):
        self.vectorizer = CountVectorizer()
        self.vectorizer.fit(corpus)

    def transform(self, text):
        return self.vectorizer.transform([text]).toarray()

class SingleHeadAttention:
    def __init__(self, input_dim, output_dim):
        self.Wq = np.random.rand(input_dim, output_dim)
        self.Wk = np.random.rand(input_dim, output_dim)
        self.Wv = np.random.rand(input_dim, output_dim)

    def forward(self, x):
        Q = x.dot(self.Wq)
        K = x.dot(self.Wk)
        V = x.dot(self.Wv)

        scores = np.dot(Q, K.T) / np.sqrt(Q.shape[-1])
        attention_weights = self.softmax(scores)
        output = np.dot(attention_weights, V)
        return output

    def softmax(self, x):
        e_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
        return e_x / e_x.sum(axis=-1, keepdims=True)

class SimpleNeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        self.W1 = np.random.rand(input_size, hidden_size)
        self.b1 = np.zeros(hidden_size)
        self.W2 = np.random.rand(hidden_size, output_size)
        self.b2 = np.zeros(output_size)

    def forward(self, x):
        hidden = self.sigmoid(np.dot(x, self.W1) + self.b1)
        output = self.sigmoid(np.dot(hidden, self.W2) + self.b2)
        return output

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

# Example Usage
text_processor = TextProcessor()
corpus = ["My name is Sasha", "Your name is John", "What is your name?"]
text_processor.fit_vectorizer(corpus)

for sentence in corpus:
    processed_sentence = text_processor.preprocess(sentence)
    print(f"Processed: {processed_sentence}")
    vectorized_sentence = text_processor.transform(sentence)
    print(f"Vectorized: {vectorized_sentence}")

# Example of using attention
attention = SingleHeadAttention(input_dim=10, output_dim=5)
x = np.random.rand(2, 10)  # Example input with batch size 2 and feature size 10
attention_output = attention.forward(x)
print(f"Attention Output: {attention_output}")

# Simple Neural Network Example
nn = SimpleNeuralNetwork(input_size=10, hidden_size=5, output_size=3)
nn_output = nn.forward(x)
print(f"Neural Network Output: {nn_output}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Processed: ['my', 'name', 'is', 'sasha']
Vectorized: [[1 0 1 1 1 0 0]]
Processed: ['your', 'name', 'is', 'john']
Vectorized: [[1 1 0 1 0 0 1]]
Processed: ['what', 'is', 'your', 'name']
Vectorized: [[1 0 0 1 0 1 1]]
Attention Output: [[2.30215297 0.99614934 3.68678039 2.24913376 3.79272347]
 [2.30268349 0.99451035 3.68990265 2.25068694 3.8014551 ]]
Neural Network Output: [[0.87960231 0.91520589 0.81668954]
 [0.8850876  0.9196608  0.82265472]]
